In [1]:
import getpass
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
load_dotenv()
from langchain_pinecone import PineconeVectorStore
from pprint import pprint
from langchain.prompts import ChatPromptTemplate




/Users/nishanshehadeh/opt/anaconda3/envs/WTP/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["PINECONE_API_KEY"] = os.getenv('PINECONE_API_KEY')

llm = ChatOpenAI(model="gpt-4o")

index_name = "langchain-index"
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

# Make actual retriever 
vector_store = PineconeVectorStore(index_name=index_name, embedding=embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
    )
template = """Answer the question using the context. Give a one line summary of the context metadata used at the end of your answer:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pprint(rag_chain.invoke("What are Biden's tax policies for young adults"))
template = """Answer the question using the context. Give a one line summary of the context metadata used at the end of your answer:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

pprint(rag_chain.invoke("What are Biden's tax policies for young adults"))

("Biden's tax policies do not specifically mention young adults; however, his "
 'budget aims to extend all middle-class tax cuts and promises not to raise '
 'taxes on anyone earning less than $400,000 per year.\n'
 '\n'
 'Summary of context metadata: Various documents from the White House briefing '
 "room discussing President Biden's budget, tax policies, and economic agenda.")
